# Road Rage: Finding the Ideal Speed Limit

### Assumptions
* Drivers want to go up to 120 km/hr.
* The average car is 5 meters long.
* Drivers want at least a number of meters equal to their speed in meters/second between them and the next car.
* Drivers will accelerate 2 m/s<sup>2</sup> up to their desired speed as long as they have room to do so.
* If another car is too close, drivers will match that car's speed until they have room again.
* If a driver would hit another car by continuing, they stop.
* Drivers will randomly (10% chance each second) slow by 2 m/s.
* This section of road is one lane going one way.
* Assume that drivers enter the road at the speed they left.
* Simulation starts with 30 cars per kilometer, evenly spaced.

## Normal Mode
We have a 1 kilometer section of road being built and do not know what the speed limit should be. This notebook simulates the 1 kilometer of road. Even though this road is not circular, the simulation treats it as such in order to generate a continuous flow of traffic.

In [24]:
from copy import deepcopy
import math
import matplotlib.pyplot as plt
import numpy as np
import random
from traffic_lib import *
%matplotlib inline

In [72]:
class Car:
    """
    Responsibilities:
    - Know speed (in m/s)
    - Know distance to driver ahead
    - Keep distance from driver ahead
    - Accelerate if possible
    - Match speed of those ahead if within safety zone
    - Stop if new location would result in crash (0 distance to car ahead)
    """

    def __init__(self, location, gap=28, speed_limit=33, start_speed=27):
        self.location = location
        self.gap = gap
        self.desired_speed = speed_limit
        self.speed = start_speed
        self.size = 5
        self.bumper = 0
        self.update_bumper()

    def __str__(self):
        return 'Car(location={},bumper={},gap={},speed={})'.format(
                self.location, self.bumper, self.gap, self.speed)

    def __repr__(self):
        return self.__str__()

    def drive(self, car_ahead):

        if self.location + self.speed >= car_ahead.bumper:
            self.speed = 0
            self.location = car_ahead.bumper - 1
        elif self.location + self.speed > car_ahead.bumper - self.speed:
            self.speed = car_ahead.speed
        elif random.random() < 0.1:
            self.speed -= 2
            if self.speed < 0:
                self.speed = 0
        elif self.speed < self.desired_speed:
            self.speed += 2
            if self.speed > self.desired_speed:
                self.speed = self.desired_speed

        self.location += self.speed
        if self.location >= 1000:
            print("I'm off the road, man!")
            return 'off_the_road'
        else:
            return 'on_the_road'

    def update_bumper(self):
        if self.location - (self.size - 1) < 1000:
            self.bumper = self.location - (self.size - 1)
        else:
            self.bumper = self.location - 1000 - (self.size - 1)

In [76]:
class Road:
    """
    Responsibilities:
    - Hold length of road
    - Keep a list of vehicles on road
        - Initialize with number of cars
        - (1000 - sum(vehicle.size)) // len(self.vehicles)
    - Hold potential for slowdown
    
    Collaborators:
    - Car
    """
    def __init__(self):
        self.total_vehicle_space = (30 * 5)
        self.initial_gap = int((1000 - self.total_vehicle_space) / 30)
        self.vehicles = [Car((4 + int(33.333333333*n)), self.initial_gap) for n in range(30)]
        self.vehicles[-1].gap = (1000 - self.vehicles[-1].location)

In [81]:
class HighwaySim:
    """
    Responsibilities:
    - Have road place cars at beginning of road when they reach the end
    - Keep track of time (seconds)
    - Step through time (ticks)
        - For each car on Road, tell car behind new situation and allow it to react
    - Report stats; return traffic jam status
    - Pop, Append

    Collaborators:
    - Car
    - Road
    """

    def __init__(self):
        self.road = Road()
        self.ticks = 0
        self.is_traffic = []

    def run_sim(self, duration=1):
        while self.ticks < duration:
            self.iterate()
            self.ticks += 1
        if self.is_traffic.count(True) > 0:
            return True
        else:
            return False

    def iterate(self):
        off_the_road = []
        num_cars = len(self.road.vehicles)
        for idx in range(num_cars):
            v = self.road.vehicles[- idx - 1]
            if idx > 0:
                car_ahead = deepcopy(self.road.vehicles[- idx])
                if car_ahead.location < v.location:
                    car_ahead.location + 1000
                    car_ahead.update_bumper()
            else:
                car_ahead = deepcopy(self.road.vehicles[0])
                car_ahead.location += 1000
                if car_ahead.gap < car_ahead.speed:
                    car_ahead.location += car_ahead.gap
                else:
                    car_ahead.location += car_ahead.speed
                car_ahead.bumper = car_ahead.location - (car_ahead.size - 1)

            if v.drive(car_ahead) == 'off the road':
                off_the_road.append(- idx - 1)
            
            if car_ahead.bumper - v.location > 0:
                v.gap = car_ahead.bumper - v.location
            else:
                v.gap = car_ahead.bumper + 1000 - v.location
                
            v.update_bumper()

            if v.speed == 0:
                self.is_traffic.append(True)
            else:
                self.is_traffic.append(False)

        while len(off_the_road) > 0:
            off_car = self.road.vehicles.pop(-1)
            off_car.location -= 1000
            if off_car.location >= off_car.size - 1:
                off_car.update_bumper()
            else:
                off_car.bumper = off_car.location - (off_car.size -1) + 1000
            self.road.vehicles.insert(0, off_car)
            off_the_road.pop(0)

In [82]:
sim1 = HighwaySim()
sim1.run_sim(1)
sim1.road.vehicles

[Car(location=33,bumper=29,gap=29,speed=29),
 Car(location=66,bumper=62,gap=29,speed=29),
 Car(location=99,bumper=95,gap=29,speed=29),
 Car(location=132,bumper=128,gap=26,speed=29),
 Car(location=162,bumper=158,gap=33,speed=25),
 Car(location=199,bumper=195,gap=29,speed=29),
 Car(location=232,bumper=228,gap=30,speed=29),
 Car(location=266,bumper=262,gap=29,speed=29),
 Car(location=299,bumper=295,gap=29,speed=29),
 Car(location=332,bumper=328,gap=26,speed=29),
 Car(location=362,bumper=358,gap=33,speed=25),
 Car(location=399,bumper=395,gap=29,speed=29),
 Car(location=432,bumper=428,gap=30,speed=29),
 Car(location=466,bumper=462,gap=29,speed=29),
 Car(location=499,bumper=495,gap=29,speed=29),
 Car(location=532,bumper=528,gap=30,speed=29),
 Car(location=566,bumper=562,gap=29,speed=29),
 Car(location=599,bumper=595,gap=29,speed=29),
 Car(location=632,bumper=628,gap=30,speed=29),
 Car(location=666,bumper=662,gap=25,speed=29),
 Car(location=695,bumper=691,gap=29,speed=25),
 Car(location=728,b

In [6]:
def highway_trials(num_trials=1000):
    """
    Run num_trials of run_sim
    """
    trial_stats = []
    for _ in range(num_trials):
        trial_data = HighwaySim()
        trial_stats.append(trial_data)
    return trial_stats

In [7]:
highway_1000_data = highway_trials()

In [8]:
print(highway_1000_data.count(True))

0
